In [6]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3 as db

In [11]:
#Get sample data
df = pd.read_csv(r'C:\Projects\Strava\RideData_20150628145227.csv')
df.head(1)

,act_startDate,timestamp,time,act_id,act_name,distance,altitude,velocity_smooth,moving,grade_smooth,timestamp2,lat,long
0,2015-06-28 11:36:16,2015-06-28 11:36:16+00:00,0,334723309,Morning Ride,0.0,180.2,0.0,False,0.0,2015-06-28 11:36:16,44.615305,-88.039688


In [12]:
#Create our test database for upserts
conn = db.connect('test_upsert.db')
engine = create_engine(r'sqlite:///test_upsert.db')

In [13]:
#Create a table using pandas to_sql
df.to_sql('test_upsert', engine, if_exists='append')

In [16]:
#Check that the table exists and there is data in it
df_in_db = pd.read_sql_query('SELECT * FROM test_upsert', engine)
df_in_db.head(1)

,index,act_startDate,timestamp,time,act_id,act_name,distance,altitude,velocity_smooth,moving,grade_smooth,timestamp2,lat,long
0,0,2015-06-28 11:36:16,2015-06-28 11:36:16+00:00,0,334723309,Morning Ride,0.0,180.2,0.0,0,0.0,2015-06-28 11:36:16,44.615305,-88.039688


In [19]:
#Now lets bring in some new data to insert, along with the same old data
df2 = pd.concat([pd.read_csv(r'C:\Projects\Strava\RideData_20150628145227.csv'),
               pd.read_csv(r'C:\Projects\Strava\RideData_20150628145421.csv')])
df2.head(1)

,act_id,act_name,act_startDate,altitude,distance,grade_smooth,lat,long,moving,time,timestamp,timestamp2,velocity_smooth
0,334723309,Morning Ride,2015-06-28 11:36:16,180.2,0.0,0.0,44.615305,-88.039688,False,0,2015-06-28 11:36:16+00:00,2015-06-28 11:36:16,0.0


In [22]:
#First let's get the length of both dataframe
len_df2 = df2.shape[0]
len_df_in_db = df_in_db.shape[0]
print ('new df is %s rows, and data in db is %s rows') %(len_df2, len_df_in_db)

new df is 11806 rows, and data in db is 5903 rows


In [24]:
#Now let's find out what rows are duplicates using a combination of full outer join and select where is not null
df_all = pd.merge(df2, df_in_db, how='outer')
df_all.describe()

,act_id,altitude,distance,grade_smooth,lat,long,time,velocity_smooth,index
count,11806.0,11806.000000,11806.000000,11806.000000,11806.000000,11806.000000,11806.000000,11806.000000,5903.000000
mean,334723309.0,198.998577,22510.749907,0.106827,44.597998,-88.086968,3357.149077,7.784686,2951.000000
std,0.0,18.299093,13539.992780,2.353318,0.019859,0.039451,1775.176818,1.413113,1704.193651
min,334723309.0,176.700000,0.000000,-19.500000,44.572781,-88.150414,0.000000,0.000000,0.000000
25%,334723309.0,179.800000,10329.525000,-0.700000,44.583538,-88.119510,1831.250000,7.000000,1475.500000
50%,334723309.0,200.300000,22122.000000,0.000000,44.591711,-88.095911,3307.000000,7.800000,2951.000000
75%,334723309.0,215.900000,34199.150000,0.700000,44.611576,-88.041069,4909.750000,8.500000,4426.500000
max,334723309.0,235.500000,46258.200000,22.500000,44.643393,-88.015313,6545.000000,12.900000,5902.000000
